<a href="https://colab.research.google.com/github/victormurcia/VCHAMPS/blob/main/Mapping_Encounter_IDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#General utilities
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm  # Import tqdm for the progress bar
import glob,shutil,os,warnings,math,time,sys,re
from typing import List
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

#For converting states to their abbreviations
#!pip install us
#import us

#For performing UTC normalization on datetime columns based on the STATE column
import pytz

#For Slider viz
import ipywidgets as widgets
from IPython.display import display, clear_output,HTML

#For EDA
#!pip install dataprep
#from dataprep.eda import create_report

#Enable data to be extracted and downloaded from my Google Drive
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Specify the path to the desired directory
directory_path = r'/content/drive/MyDrive/VCHAMPS - Train Cleaned'

# Change the current working directory to the desired directory
os.chdir(directory_path)

# Verify the current working directory
cwd = os.getcwd()

print(f"Current working directory: {cwd}")

Current working directory: /content/drive/MyDrive/VCHAMPS - Train Cleaned


In [3]:
def load_parquet_files(folder_path):
    # Get a list of all files in the current directory
    all_files = [f.path for f in os.scandir(folder_path) if f.is_file()]

    # Get a list of all subdirectories (dataset folders) in the current directory
    dataset_folders = [f.path for f in os.scandir(folder_path) if f.is_dir()]

    # Initialize an empty list to store the DataFrames
    dataframes = []

    # Load Parquet files in the current directory
    for file in all_files:
        if file.endswith('.parquet'):
            df = dd.read_parquet(file, engine='pyarrow')
            dataframes.append(df)

    # Load Parquet files in the subfolders
    for folder in dataset_folders:
      if folder.endswith('.parquet'):
        df = dd.read_parquet(folder + '/*.parquet', engine='pyarrow')
        dataframes.append(df)

    return dataframes

dataframes = load_parquet_files(cwd)

# Generate Encounter IDs
Do this based on the inpatient_admisssions, ed_visits,and outpatient visits files.

I'll either use a Hash or a UUID to define the Encounter ID

In [4]:
import uuid
# Define a dictionary to store generated UUIDs
uuid_dict = {}

# Define a custom function to generate UUIDs and ensure uniqueness
def generate_uuid(row, df_val):
    if df_val == 1:
      columns = ['Internalpatientid', 'Ed visit start date', 'Discharge date ed']
    elif df_val == 2:
      columns = ['Internalpatientid', 'Admission date', 'Discharge date']
    elif df_val == 3:
      columns = ['Internalpatientid', 'Visit start date']

    unique_values = tuple(row[column] for column in columns)
    unique_values += (df_val,)  # Append the additional parameter to the unique_values tuple
    key = str(unique_values)

    # Check if UUID already exists in the dictionary
    if key in uuid_dict:
        return uuid_dict[key]

    # Generate a new UUID and store it in the dictionary
    new_uuid = str(uuid.uuid5(uuid.NAMESPACE_OID, key))
    uuid_dict[key] = new_uuid
    return new_uuid

In [5]:
import hashlib

def generate_hash_id(row, df_val):
    if df_val == 1:
        columns = ['Internalpatientid', 'Ed visit start date', 'Discharge date ed']
    elif df_val == 2:
        columns = ['Internalpatientid', 'Admission date', 'Discharge date']
    elif df_val == 3:
        columns = ['Internalpatientid', 'Visit start date']

    data = ''.join(str(row[column]) for column in columns)
    data += str(df_val)

    hash_object = hashlib.sha256(data.encode())
    hash_id = hash_object.hexdigest()

    return hash_id

In [6]:
#Load the Dataframes
ed_visits_df            = dd.read_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/ed_visits.parquet/*.parquet')
inpatient_admissions_df = dd.read_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/inpatient_admissions.parquet')
outpatient_visits_df    = dd.read_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/outpatient_visits.parquet/*.parquet')

I decided to use UUID instead of Hash since UUIDs used far less memory

In [7]:
# Assign UUIDs to each row
ed_visits_df['Encounter ID'] = ed_visits_df.apply(generate_uuid, args=(1,), axis=1, meta=(None, 'object'))
inpatient_admissions_df['Encounter ID'] = inpatient_admissions_df.apply(generate_uuid, args=(2,), axis=1, meta=(None, 'object'))
outpatient_visits_df['Encounter ID'] = outpatient_visits_df.apply(generate_uuid, args=(3,), axis=1, meta=(None, 'object'))

In [15]:
ed_visits_df.head()

,Internalpatientid,Age at ed visit,Ed visit start date,Discharge date ed,Died during ed visit,CV diagnosis,code,cc Status,Encounter ID
0,101689,64,2021-08-26 00:53:30,2021-08-26 04:24:27,0,1,I502,NCC,2afc637f-1e93-5ce7-b817-0784cebc77a1
1,101689,64,2021-08-26 00:53:30,2021-08-26 04:24:27,0,1,N189,NCC,2afc637f-1e93-5ce7-b817-0784cebc77a1
2,107210,71,2022-05-18 11:21:28,2022-05-18 18:30:21,0,0,A419,MCC,0d3f5146-915f-5e92-b3e0-0ea9f4aa0141
3,107210,71,2022-05-18 11:21:28,2022-05-18 18:30:21,0,0,I959,NCC,0d3f5146-915f-5e92-b3e0-0ea9f4aa0141
4,107866,61,2017-06-26 15:41:11,2017-06-26 23:33:50,0,0,R918,NCC,efb622f7-8c68-5b26-ae9f-09147ff0dce7


In [18]:
inpatient_admissions_df.head()

,Internalpatientid,Age at admission,Admission date,Discharge date,Admitting unit service,Discharging unit service,Admitting specialty,Discharging specialty,Died during admission,Outpatientreferralflag,...,cc Status_CC,cc Status_MCC,cc Status_NCC,Discharge disposition_Death with autopsy,Discharge disposition_Death without autopsy,Discharge disposition_Irregular,Discharge disposition_NBC or while ASIH,Discharge disposition_Regular,Discharge disposition_Transfer,Encounter ID
0,10,66,2015-11-28 17:41:09,2015-11-29 01:43:14,NON-COUNT,NON-COUNT,DRUG DEPENDENCE TRMT UNIT,MEDICAL OBSERVATION,0,0,...,0,1,0,0,0,0,0,1,0,974dbd04-1e56-59c6-a720-b8784dc604b2
1,10,66,2015-11-28 17:41:09,2015-11-29 01:43:14,NON-COUNT,NON-COUNT,DRUG DEPENDENCE TRMT UNIT,MEDICAL OBSERVATION,0,0,...,0,0,1,0,0,0,0,1,0,974dbd04-1e56-59c6-a720-b8784dc604b2
2,100001,84,2009-10-01 21:19:50,2009-10-04 16:51:33,MEDICINE,MEDICINE,PSYCHIATRIC MENTALLY INFIRM,GENERAL(ACUTE MEDICINE),0,1,...,0,1,0,0,0,0,0,1,0,c86658fc-cb34-536f-bf7c-a6a05ad131cc
3,100001,84,2009-10-01 21:19:50,2009-10-04 16:51:33,MEDICINE,MEDICINE,PSYCHIATRIC MENTALLY INFIRM,GENERAL(ACUTE MEDICINE),0,1,...,0,0,1,0,0,0,0,1,0,c86658fc-cb34-536f-bf7c-a6a05ad131cc
4,100001,85,2010-11-10 04:32:39,2010-11-19 08:49:45,SURGERY,SURGERY,SUBSTANCE ABUSE RES TRMT PROG,ORTHOPEDIC,0,0,...,0,0,1,0,0,0,0,1,0,9b0880c7-906a-5869-9bbb-698c6183bdec


In [19]:
outpatient_visits_df.head()

,Internalpatientid,Age at visit,Visit start date,Visit End Date,Stop code,Agentorangeflag,Combatflag,Ionizingradiationflag,Serviceconnectedflag,Swasiaconditionsflag,diagnosis,code,cc Status,Encounter ID
0,10037,57,2014-06-17 11:56:20,2014-06-17 13:56:20,EKG,NaN,NaN,NaN,NaN,NaN,other hypertrophic cardiomyopathy,I422,CC,7ca25b58-b33a-57a1-83c1-7fab40faaaff
0,65015,67,2018-06-21 01:25:05,2018-06-21 03:25:05,OPTOMETRY,NaN,NaN,NaN,No,NaN,type 2 diabetes mellitus without complications,E119,NCC,2631ee2b-20a4-5b72-bc1f-ff7c8e4589dd
0,6139,74,2022-07-14 21:44:44,2022-07-14 23:44:44,HT NON-VIDEO MONITORING,NaN,NaN,NaN,Yes,NaN,type 2 diabetes mellitus with unspecified comp...,E118,NCC,2c2ca169-48b0-5fce-bfe5-d0786a534457
0,18667,89,2021-03-22 11:41:03,2021-03-22 13:41:03,HOSPITAL IN HOME,NaN,NaN,NaN,NaN,NaN,chronic atrial fibrillation,I482,NCC,2fc52270-9dde-554c-a79d-81022ad98b67
0,77347,64,2019-07-03 13:45:28,2019-07-03 15:45:28,PRIMARY CARE/MEDICINE,NaN,NaN,NaN,Yes,NaN,kidney transplant status,Z940,CC,0afb8078-9f9b-50cf-b0d8-397ad8b01805


Now that each of the visits have a UUID associated with it, I can start mapping the other dataframes to them

In [8]:
ed_visits_df = ed_visits_df.compute()
inpatient_admissions_df = inpatient_admissions_df.compute()
outpatient_visits_df = outpatient_visits_df.compute()

In [49]:
ed_visits_df

,Internalpatientid,Age at ed visit,Ed visit start date,Discharge date ed,Died during ed visit,CV diagnosis,code,cc Status,Encounter ID
0,101689,64,2021-08-26 00:53:30,2021-08-26 04:24:27,0,1,I502,NCC,2afc637f-1e93-5ce7-b817-0784cebc77a1
1,101689,64,2021-08-26 00:53:30,2021-08-26 04:24:27,0,1,N189,NCC,2afc637f-1e93-5ce7-b817-0784cebc77a1
2,107210,71,2022-05-18 11:21:28,2022-05-18 18:30:21,0,0,A419,MCC,0d3f5146-915f-5e92-b3e0-0ea9f4aa0141
3,107210,71,2022-05-18 11:21:28,2022-05-18 18:30:21,0,0,I959,NCC,0d3f5146-915f-5e92-b3e0-0ea9f4aa0141
4,107866,61,2017-06-26 15:41:11,2017-06-26 23:33:50,0,0,R918,NCC,efb622f7-8c68-5b26-ae9f-09147ff0dce7
...,...,...,...,...,...,...,...,...,...
1099360,9883,90,2020-02-20 21:23:37,2020-02-20 22:56:30,0,0,W108,NCC,da0bbfb2-28fc-5b3a-afcb-2642eee54c6f
1099422,99235,87,2016-02-25 01:22:23,2016-02-25 06:33:45,0,0,R197,NCC,7c5682aa-be6e-5fca-8c73-ec208a0ac60f
1099538,99902,66,2018-09-28 23:21:41,2018-09-28 23:56:55,0,0,M796,NCC,15df3aab-aa5b-51fa-b993-2d1cad1fa444
1099548,99934,84,2022-07-22 09:19:20,2022-07-22 09:42:57,0,0,M796,NCC,b3f5f12b-57fd-5978-842e-8f8f8c293522


In [31]:
inpatient_admissions_df

,Internalpatientid,Age at admission,Admission date,Discharge date,Admitting unit service,Discharging unit service,Admitting specialty,Discharging specialty,Died during admission,Outpatientreferralflag,...,cc Status_CC,cc Status_MCC,cc Status_NCC,Discharge disposition_Death with autopsy,Discharge disposition_Death without autopsy,Discharge disposition_Irregular,Discharge disposition_NBC or while ASIH,Discharge disposition_Regular,Discharge disposition_Transfer,Encounter ID
0,10,66,2015-11-28 17:41:09,2015-11-29 01:43:14,NON-COUNT,NON-COUNT,DRUG DEPENDENCE TRMT UNIT,MEDICAL OBSERVATION,0,0,...,0,1,0,0,0,0,0,1,0,974dbd04-1e56-59c6-a720-b8784dc604b2
1,10,66,2015-11-28 17:41:09,2015-11-29 01:43:14,NON-COUNT,NON-COUNT,DRUG DEPENDENCE TRMT UNIT,MEDICAL OBSERVATION,0,0,...,0,0,1,0,0,0,0,1,0,974dbd04-1e56-59c6-a720-b8784dc604b2
2,100001,84,2009-10-01 21:19:50,2009-10-04 16:51:33,MEDICINE,MEDICINE,PSYCHIATRIC MENTALLY INFIRM,GENERAL(ACUTE MEDICINE),0,1,...,0,1,0,0,0,0,0,1,0,c86658fc-cb34-536f-bf7c-a6a05ad131cc
3,100001,84,2009-10-01 21:19:50,2009-10-04 16:51:33,MEDICINE,MEDICINE,PSYCHIATRIC MENTALLY INFIRM,GENERAL(ACUTE MEDICINE),0,1,...,0,0,1,0,0,0,0,1,0,c86658fc-cb34-536f-bf7c-a6a05ad131cc
4,100001,85,2010-11-10 04:32:39,2010-11-19 08:49:45,SURGERY,SURGERY,SUBSTANCE ABUSE RES TRMT PROG,ORTHOPEDIC,0,0,...,0,0,1,0,0,0,0,1,0,9b0880c7-906a-5869-9bbb-698c6183bdec
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085813,99975,89,2018-01-05 00:59:05,2018-01-06 23:18:39,NON-COUNT,NON-COUNT,PODIATRY,MEDICAL OBSERVATION,0,0,...,0,0,1,0,0,0,0,1,0,95223157-498c-5de5-a061-13d1e9ae985a
1085814,99986,73,2019-06-26 19:47:10,2019-06-29 21:18:00,MEDICINE,MEDICINE,INTERMEDIATE MEDICINE,GENERAL(ACUTE MEDICINE),0,1,...,0,0,1,0,0,0,0,1,0,bd343752-5059-5181-81ad-dd066c9179c3
1085815,99986,73,2019-06-26 19:47:10,2019-06-29 21:18:00,MEDICINE,MEDICINE,INTERMEDIATE MEDICINE,GENERAL(ACUTE MEDICINE),0,1,...,0,0,1,0,0,0,0,1,0,bd343752-5059-5181-81ad-dd066c9179c3
1085816,99994,83,2016-08-13 20:09:52,2016-08-15 16:43:07,SURGERY,MEDICINE,INTERMEDIATE MEDICINE,GENERAL(ACUTE MEDICINE),0,1,...,1,0,0,0,0,0,0,1,0,a0b6452b-5012-5b06-81f3-150b3a92b656


In [40]:
outpatient_visits_df

,Internalpatientid,Age at visit,Visit start date,Visit End Date,Stop code,Agentorangeflag,Combatflag,Ionizingradiationflag,Serviceconnectedflag,Swasiaconditionsflag,diagnosis,code,cc Status,Encounter ID
0,10037,57,2014-06-17 11:56:20,2014-06-17 13:56:20,EKG,NaN,NaN,NaN,NaN,NaN,other hypertrophic cardiomyopathy,I422,CC,7ca25b58-b33a-57a1-83c1-7fab40faaaff
0,65015,67,2018-06-21 01:25:05,2018-06-21 03:25:05,OPTOMETRY,NaN,NaN,NaN,No,NaN,type 2 diabetes mellitus without complications,E119,NCC,2631ee2b-20a4-5b72-bc1f-ff7c8e4589dd
0,6139,74,2022-07-14 21:44:44,2022-07-14 23:44:44,HT NON-VIDEO MONITORING,NaN,NaN,NaN,Yes,NaN,type 2 diabetes mellitus with unspecified comp...,E118,NCC,2c2ca169-48b0-5fce-bfe5-d0786a534457
0,18667,89,2021-03-22 11:41:03,2021-03-22 13:41:03,HOSPITAL IN HOME,NaN,NaN,NaN,NaN,NaN,chronic atrial fibrillation,I482,NCC,2fc52270-9dde-554c-a79d-81022ad98b67
0,77347,64,2019-07-03 13:45:28,2019-07-03 15:45:28,PRIMARY CARE/MEDICINE,NaN,NaN,NaN,Yes,NaN,kidney transplant status,Z940,CC,0afb8078-9f9b-50cf-b0d8-397ad8b01805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054843,10037,57,2014-05-07 15:00:57,2014-05-07 17:00:57,TELEPHONE/MEDICINE,NaN,NaN,NaN,NaN,NaN,type 2 diabetes mellitus with ophthalmic compl...,E113,NCC,0f2852c3-7975-5250-8a19-cc0fc47eba3a
1054845,10037,57,2014-05-16 06:07:53,2014-05-16 08:07:53,PRIMARY CARE/MEDICINE,NaN,NaN,NaN,NaN,NaN,dizziness and giddiness,R42,NCC,5c42cde4-5596-5ec4-944e-0608829d5835
1054846,10037,57,2014-05-16 06:07:53,2014-05-16 08:07:53,PRIMARY CARE/MEDICINE,NaN,NaN,NaN,NaN,NaN,type 2 diabetes mellitus with ophthalmic compl...,E113,NCC,5c42cde4-5596-5ec4-944e-0608829d5835
1054847,10037,57,2014-06-17 11:56:20,2014-06-17 13:56:20,PRIMARY CARE/MEDICINE,NaN,NaN,NaN,NaN,NaN,fracture of other and unspecified finger(s),S626,NCC,7ca25b58-b33a-57a1-83c1-7fab40faaaff


In [9]:
#Make a list of all use encounter IDs. Will be used to ensure uniqueness later
encounter_ids = []
encounter_ids.extend(ed_visits_df['Encounter ID'].tolist())
encounter_ids.extend(inpatient_admissions_df['Encounter ID'].tolist())
encounter_ids.extend(outpatient_visits_df['Encounter ID'].tolist())

In [15]:
def map_encounter_id(row, age_col, date_col):
    patient_id    = row['Internalpatientid']
    patient_age   = row[age_col]
    date_to_match = row[date_col]

    #1. Check for matches in the ed_visits first
    # Filter the first dataframe for matching patient ID and age conditions
    filtered_df = ed_visits_df[(ed_visits_df['Internalpatientid'] == patient_id) & (ed_visits_df['Age at ed visit'] <= patient_age)]
    # Find the first instance where date_to_match is between the start and end dates
    filtered_df = filtered_df[(filtered_df['Ed visit start date'] <= date_to_match) & (filtered_df['Discharge date ed'] >= date_to_match)]

    if len(filtered_df) > 0:
        #print('Found match in ed_visits for ',patient_id)
        return filtered_df['Encounter ID'].iloc[0]
    else:
        #2. Check for matches in the inpatient_visits if no match is found in ed_visits
        filtered_df = inpatient_admissions_df[(inpatient_admissions_df['Internalpatientid'] == patient_id) & (inpatient_admissions_df['Age at admission'] <= patient_age)]
         # Find the first instance where date_to_match is between the start and end dates
        filtered_df = filtered_df[(filtered_df['Admission date'] <= date_to_match) & (filtered_df['Discharge date'] >= date_to_match)]
        if len(filtered_df) > 0:
          #print('Found match in inpatient_visits for ',patient_id)
          return filtered_df['Encounter ID'].iloc[0]
        else:
          #3. Check for matches in the outpatient_visits if no match is found in inpatient_visits
          filtered_df = outpatient_visits_df[(outpatient_visits_df['Internalpatientid'] == patient_id) & (outpatient_visits_df['Age at visit'] <= patient_age)]
          # Find the first instance where date_to_match is between the start and end dates
          filtered_df = filtered_df[(filtered_df['Visit start date'] <= date_to_match) & (filtered_df['Visit End Date'] >= date_to_match)]
          if len(filtered_df) > 0:
            #print('Found match in outpatient_visits for ',patient_id)
            return filtered_df['Encounter ID'].iloc[0]
          else:
            #print('No match found. Producing unique Encounter ID')
            return str(uuid.uuid4())
    return None

In [67]:
small_death_df['Encounter ID'] = small_death_df.apply(lambda row: map_encounter_id(row, 'Age at death', 'Death date'), axis=1)

In [68]:
#I tested the function in a subset just to ensure proper operation and it works well and is fairly fast.
small_death_df

,Internalpatientid,Age at death,Death date,Encounter ID
0,100036,72,2015-01-18 20:49:10,6dadd260-ca9b-472e-bb62-f4bb2860271f
1,100037,75,2008-10-16 15:14:26,dab2ca47-25fe-4412-9e34-227febe7b029
2,100045,85,2019-01-26 11:33:52,78fb7dc0-27bf-42c8-85e6-c34ff313081f
3,100095,84,2004-08-10 21:49:35,9faaaf6d-e96d-45cc-8121-ec159aceb691
4,10013,68,2007-06-11 00:17:06,7a915113-5c57-4cc3-b503-d170915768f0
...,...,...,...,...
95,102820,82,2001-03-05 01:52:28,6a204164-030d-40c4-b44f-27ff1b893e9a
96,102875,91,2018-10-19 23:59:54,445a1c16-c74f-48f1-9704-f924c006cae6
97,102878,68,2011-03-30 06:54:04,04047807-7d4a-4516-854b-af18bf9e2018
98,102924,100,2015-01-22 02:06:58,01dbd389-2e8c-4789-961b-6cb407f48f6a


# Mapping Death DF

In [7]:
death_df = dd.read_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/death.parquet')

In [10]:
death_df = death_df.compute()

In [69]:
death_df['Encounter ID'] = death_df.apply(lambda row: map_encounter_id(row, 'Age at death', 'Death date'), axis=1)

In [70]:
# Save the Dask DataFrame as Parquet
death_df.to_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned-Mapped/death.parquet', engine='pyarrow')

# Mapping Immunizations DF

In [ ]:
immunizations_df = dd.read_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/immunizations.parquet')
immunizations_df = immunizations_df.compute()
immunizations_df.columns

In [ ]:
immunizations_df['Encounter ID'] = immunizations_df.apply(lambda row: map_encounter_id(row, 'Age at location', 'Location start date'), axis=1)

# Mapping Inpatient Locations DF

In [12]:
inpatient_location_df = dd.read_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/inpatient_location.parquet')
inpatient_location_df = inpatient_location_df.compute()
inpatient_location_df.columns

Index(['Internalpatientid', 'Age at location', 'Location start date',
       'Location end date', 'Service', 'Died at location'],
      dtype='object')

In [13]:
inpatient_location_df

,Internalpatientid,Age at location,Location start date,Location end date,Service,Died at location
0,100005,66,2002-09-09 01:24:31,2002-09-12 01:00:40,MEDICINE,0
1,100008,70,2019-06-04 05:46:59,2019-06-05 10:07:31,SURGERY,0
2,100015,55,2001-04-01 01:32:23,2001-04-02 20:04:22,MEDICINE,0
3,100015,56,2001-07-29 20:19:52,2001-07-30 22:42:13,MEDICINE,0
5,100026,61,2017-04-22 12:58:18,2017-04-25 03:43:53,MEDICINE,0
...,...,...,...,...,...,...
850032,99974,53,2003-08-25 17:35:48,2003-08-31 15:18:56,MEDICINE,0
850033,99981,67,2009-08-21 05:12:20,2009-08-24 07:10:02,SURGERY,0
850034,99983,44,2006-04-22 23:06:15,2006-04-29 13:25:45,DOMICILIARY,0
850035,9999,61,2013-09-14 10:54:27,2013-09-15 06:16:09,SURGERY,0


In [16]:
inpatient_location_df['Encounter ID'] = inpatient_location_df.apply(lambda row: map_encounter_id(row, 'Age at location', 'Location start date'), axis=1)

In [17]:
inpatient_location_df

,Internalpatientid,Age at location,Location start date,Location end date,Service,Died at location,Encounter ID
0,100005,66,2002-09-09 01:24:31,2002-09-12 01:00:40,MEDICINE,0,6acbda18-63a1-5086-a5f2-64bbb198632e
1,100008,70,2019-06-04 05:46:59,2019-06-05 10:07:31,SURGERY,0,ef5e041b-4e3a-5cba-afa8-de036ba335d4
2,100015,55,2001-04-01 01:32:23,2001-04-02 20:04:22,MEDICINE,0,c90d9a73-08d3-5217-aa5f-69225425b1eb
3,100015,56,2001-07-29 20:19:52,2001-07-30 22:42:13,MEDICINE,0,bdcc756b-a0be-509c-aedd-f366602ee3a9
5,100026,61,2017-04-22 12:58:18,2017-04-25 03:43:53,MEDICINE,0,c8750041-50d2-5e6c-8587-a641615a1182
...,...,...,...,...,...,...,...
850032,99974,53,2003-08-25 17:35:48,2003-08-31 15:18:56,MEDICINE,0,89bfbaf3-d368-4ca5-b5bd-cd8f08e891b7
850033,99981,67,2009-08-21 05:12:20,2009-08-24 07:10:02,SURGERY,0,72f1a0a9-3887-5b36-9c34-7fa4bc871441
850034,99983,44,2006-04-22 23:06:15,2006-04-29 13:25:45,DOMICILIARY,0,ced196f3-133d-59a9-9040-9b3601fa2189
850035,9999,61,2013-09-14 10:54:27,2013-09-15 06:16:09,SURGERY,0,eaf7690b-8b9d-5a21-ab1b-ae476aa4b7ce


# Mapping Inpatient Specialty DF

In [ ]:
inpatient_specialty_df = dd.read_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/inpatient_specialty.parquet')
inpatient_specialty_df = inpatient_specialty_df.compute()
inpatient_specialty_df.columns

In [ ]:
inpatient_specialty_df['Encounter ID'] = inpatient_specialty_df.apply(lambda row: map_encounter_id(row, 'Age at location', 'Location start date'), axis=1)

# Mapping Measurements DF

In [ ]:
inpatient_specialty_df = dd.read_parquet('/content/drive/MyDrive/VCHAMPS - Train Cleaned/inpatient_specialty.parquet')
inpatient_specialty_df = inpatient_specialty_df.compute()
inpatient_specialty_df.columns

In [ ]:
inpatient_specialty_df['Encounter ID'] = inpatient_specialty_df.apply(lambda row: map_encounter_id(row, 'Age at location', 'Location start date'), axis=1)